In [61]:
import urllib.request
import sys
import json
import pandas as pd
import warnings
import pyodbc
from sqlalchemy import create_engine, text

# Ignore all warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', 25)

## Uploading Excel Sheet Data - Preprocessing

In [62]:
def pre_merge_stage(user_saydate):
    
    data_dpm = pd.read_excel(f'C:/SCHOOLWORK/Y3 SEM 1/DSCP/Assignment 3 - Sprint 3/Preprocessing/Cleaned Estate Office Data/DPM/{user_saydate}.xlsx')
    data_inv = pd.read_excel(f'C:/SCHOOLWORK/Y3 SEM 1/DSCP/Assignment 3 - Sprint 3/Preprocessing/Cleaned Estate Office Data/INV/{user_saydate}.xlsx')

    # Merging the dataframes
    merged_data = pd.merge(data_dpm, data_inv, on=['Date and Time', 'Location Code'], suffixes=('_dpm', '_inv'))

    return merged_data

In [63]:
def merged_preprocess(user_saydate):
    
    merged_data = pre_merge_stage(user_saydate)

    columns = ['Date and Time', 'PanelCode','BlockNo','Energy kWh_dpm', 'Expected Value kWh','Energy kWh_inv','IRR Value W/m²', 'PR %','DPMSensorIssue','INVSensorIssue']
    merged_data['BlockNo'] = merged_data['Location Code'].str.extract(r'(\d{2})')
    merged_data['SensorNo'] = merged_data['Sensor ID'].str.extract(r'(\d{2})$')
    merged_data['PanelCode'] = merged_data['BlockNo'] + '-' + merged_data['SensorNo']
    merged_data['INVSensorIssue'] = merged_data.apply(lambda row: 'Issue' if row['Energy kWh_dpm'] > row['Energy kWh_inv'] else 'No Issue', axis=1)
    sorted_data = merged_data.sort_values(by=['Date and Time', 'SensorNo'])

    cleaned_merged = sorted_data[columns]
    cleaned_merged = cleaned_merged.rename(columns={'Date and Time': 'RecordDate', 'Energy kWh_dpm': 'DPM','Expected Value kWh': 'ExpectedDPM', 'Energy kWh_inv': 'INV','IRR Value W/m²': 'IRR','PR %': 'PR',})

    return cleaned_merged

## API Weather Data - Preprocessing

In [64]:
def api_date_retrieval(data_file):
    data = pd.read_excel(f'./{data_file}.xlsx')
    data_date_amend = data.copy()
    data_date_amend['Date and Time'] = pd.to_datetime(data_date_amend['Date and Time'])

    earliest_date = data_date_amend['Date and Time'].min().date().strftime('%Y-%m-%d')
    latest_date = data_date_amend['Date and Time'].max().date().strftime('%Y-%m-%d')

    return earliest_date, latest_date

In [65]:
def api_data_retrieval(user_input):

  earliest_date, latest_date = api_date_retrieval(user_input)
  
  try: 
    url = f"https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/Clementi,Singapore/{earliest_date}/{latest_date}?unitGroup=us&key=MAEVMCSQZL8TPWXXYV5FQSLBJ&contentType=json"
    ResultBytes = urllib.request.urlopen(url)

    jsonData = json.load(ResultBytes)
          
  except urllib.error.HTTPError  as e:
    ErrorInfo= e.read().decode() 
    print('Error code: ', e.code, ErrorInfo)
    sys.exit()
  except  urllib.error.URLError as e:
    ErrorInfo= e.read().decode() 
    print('Error code: ', e.code,ErrorInfo)
    sys.exit()

  return jsonData

In [66]:
def api_data_preprocessing(ask_user_date):

    some_data = api_data_retrieval(ask_user_date)

    columns_to_extract = ['datetime', 'conditions', 'temp', 'tempmax', 'tempmin', 'precip','cloudcover','solarenergy',
                        'solarradiation', 'windspeed', 'uvindex', 'humidity', 'dew', 'pressure', 'visibility']

    weather_data = some_data['days'] if 'days' in some_data else some_data

    # Convert the list of dictionaries to a DataFrame
    df = pd.json_normalize(weather_data)
    extracted_df = df[columns_to_extract]
    extracted_df['temp'] = (extracted_df['temp'] - 32) * (5/9)
    extracted_df = extracted_df.round(2)
    extracted_df['uvindex'] = extracted_df['uvindex'].astype(int)
    extracted_df = extracted_df.rename(columns={'datetime': 'RecordDate', 'conditions': 'Conditions','temp': 'Temp', 'tempmax': 'TempMax','tempmin': 'TempMin', 'precip': 'Precipitation',
                                                'cloudcover': 'CloudCover', 'solarenergy': 'SolarEnergy','solarradiation': 'SolarRadiation', 'windspeed': 'WindSpeed','uvindex': 'UVIndex',
                                                'humidity': 'Humidity','dew': 'Dew', 'pressure': 'Pressure','visibility': 'Visibility'})

    return extracted_df

## Merging Both Datasets

In [67]:
def preprocess_data(user_types_data):
    
    main_data = merged_preprocess(user_types_data)
    weather_data = api_data_preprocessing(user_types_data)
    final_data = pd.merge(main_data, weather_data, on=['RecordDate'])

    return final_data

## Data Storage via Azure Database

In [69]:
df_to_azure = preprocess_data('Feb 2022')

server = 'dscpserver.database.windows.net'
database = 'DSCPDatabase'
username = 'admindscp'
password = 'T01b26121gH'
driver = 'ODBC Driver 17 for SQL Server'

conn_str = f'DRIVER={{{driver}}};SERVER={server};DATABASE={database};UID={username};PWD={password}'
conn = pyodbc.connect(conn_str)
engine = create_engine(f'mssql+pyodbc:///?odbc_connect={conn_str}')

table_name = 'newTable'
df_to_azure.to_sql(table_name, engine, index=False, if_exists='append')

print("DataFrame successfully written to Azure SQL Database.")

DataFrame successfully written to Azure SQL Database.
